# Image registration for Mavic 3M
As it is a Mavic 3M flight without calibration panel, only the images are co-registered and saved as 4 channel .tifs
* For the following Dataset: https://www.sciencedirect.com/science/article/pii/S2352340924004487
* Using code snippets from: https://gitlab.com/Yario/image_registration_dji_mavic_3m/-/blob/main/coregistration_mavic_3m_images.py


In [2]:
import os
import numpy as np
from glob import glob
from multiprocessing import Pool
from itertools import repeat
from PIL import Image
import piexif
import cv2
from tqdm import tqdm
import tifffile
import tifftools
from xml.etree import cElementTree as ElementTree
from tifffile import TiffFile
import xml.etree.ElementTree as ET
# Utility functions

def calibrate_image(nir_image, metadata):
    # Parse the metadata
    root = ET.fromstring(metadata["XMP"])
    xmldict = XmlDictConfig(root)
    base = xmldict['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF']['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description']
    gimbal_r = base['{http://www.dji.com/drone-dji/1.0/}GimbalReverse']
    
    # Extract necessary metadata values
    black_level = 4096  # Fixed value
    vignetting_coeffs = [float(k) for k in base['{http://www.dji.com/drone-dji/1.0/}VignettingData'].split(",")]  # Extract vignetting coefficients
    center_x = float(base['{http://www.dji.com/drone-dji/1.0/}CalibratedOpticalCenterX'])
    center_y = float(base['{http://www.dji.com/drone-dji/1.0/}CalibratedOpticalCenterY'])
    sensor_gain = float(base['{http://www.dji.com/drone-dji/1.0/}SensorGain'])
    exposure_time = float(base['{http://www.dji.com/drone-dji/1.0/}ExposureTime'])
    irradiance = float(base['{http://www.dji.com/drone-dji/1.0/}Irradiance'])  # NIR_LS * p_LSNIR
    
    # Step 1: Normalize the raw pixel values
    nir_camera = (nir_image / 65535 - black_level / 65535) * 1e6 / (sensor_gain * exposure_time)
    
    # Step 2: Apply vignetting correction
    height, width = nir_image.shape
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    r = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)
    vignetting_correction = sum([vignetting_coeffs[i] * r ** i for i in range(len(vignetting_coeffs))]) + 1.0
    nir_camera_corrected = nir_camera * vignetting_correction


    dewarp = base['{http://www.dji.com/drone-dji/1.0/}DewarpData'].split(",")
    # Step 3: Apply distortion correction
    fx = float(dewarp[0].split(";")[1])
    fy = float(dewarp[1])
    cx = float(dewarp[2])
    cy = float(dewarp[3])
    
    camera_matrix = np.array([[fx, 0, center_x + cx],
                              [0, fy, center_y + cy],
                              [0, 0, 1]])
    
    k1 = float(dewarp[-5])
    k2 = float(dewarp[-4])
    p1 = float(dewarp[-3])
    p2 = float(dewarp[-2])
    k3 = float(dewarp[-1])

    dist_coeffs = np.array([k1,k2,p1,p2,k3])
    
    # Apply distortion correction
    nir_camera_undistorted = cv2.undistort(nir_camera_corrected, camera_matrix, dist_coeffs)
    
    # Step 4: Calculate reflectance
    reflectance = nir_camera_undistorted / irradiance
    
    # Save the reflectance image
    return reflectance


def load_image(filename):
    """Load a single image with the metadata as byte dump."""
    image = np.asarray(Image.open(filename))

    t = TiffFile(filename)
    metadata = t.pages[0].tags
    

    
    # Convert tifftags to a dictionary
    metadata_dict = {tag.name: tag.value for tag in metadata.values()}
    metadata_dict["XMP"] = metadata_dict["XMP"].decode("utf-8")
    metadata_dict["ExifTag"]["FileSource"] = metadata_dict["ExifTag"]["FileSource"].decode()
    metadata_dict["ExifTag"]["SceneType"] = metadata_dict["ExifTag"]["SceneType"].decode()
    metadata_dict["GPSTag"]["GPSVersionID"] = tuple(metadata_dict["GPSTag"]["GPSVersionID"])
    metadata_dict["XPComment"] = list(metadata_dict["XPComment"])
    metadata_dict['XPKeywords'] = metadata_dict['XPKeywords'].decode()

    tags = metadata
    exif_ifd = tags.get(34665)  # Exif IFD tag
    gps_ifd = tags.get(34853)   # GPS IFD tag
    
    # Step 2: Convert tags to extratags format
    extratags = []
    subifds = []
        # Handle Exif IFD
    if exif_ifd:
        subifds.append(exif_ifd.value)
        #extratags.append((34665, 'I', 1, 0))  # Pointer to the first sub-IFD
    
    # Handle GPS IFD
    if gps_ifd:
        subifds.append(gps_ifd.value)
        #extratags.append((34853, 'I', 1, 1))  # Pointer to the second sub-IFD
    
    # Step 3: Add remaining tags to extratags
    for tag in tags.values():
        if tag.code not in [257, 258, 259, 262, 270, 273, 277, 279, 305, 34665, 34853, 256]:  # Skip Exif and GPS IFDs
            dtype = tag.dtype
            value = tag.value
    
            if isinstance(value, str):
                value = value.encode('utf-8')
    
            count = len(value) if isinstance(value, (bytes, bytearray)) else 1
            extratags.append((tag.code, dtype, count, value))
    
    return image, (extratags,subifds,metadata_dict)
    
def process_xml_dict_3m(xml_string):
    root = ET.fromstring(xml_string)
    xmldict = XmlDictConfig(root)
    return xmldict
    
def save_image(filename, image, metadata=None):
    """Save the image as a 4-channel TIFF using tifffile."""
    #tifffile.imwrite(filename, image, photometric='rgb',extratags=metadata[0],subifds = metadata[1])
    if metadata is not None:
        extratags = metadata[0]
        subifds = metadata[1]
    else:
        extratags = None
        subifds= None
        
    with tifffile.TiffWriter(filename) as tiff:
        tiff.write(
            image,
            photometric='rgb',
            extratags=extratags,
            subifds=subifds
        )

# Create a directory to save homographies
os.makedirs("homographies", exist_ok=True)


def align_images(image1, image2, band_name, green_image_path, set_title = "set"):
    """Align two images using ORB keypoints and cached affine transformation with quantization."""
    # Create a unique transformation filename
    base_name = os.path.basename(green_image_path).replace("_G.TIF", "")
    affine_filename = f"homographies/{band_name}_{set_title}_affine.npy"

    # Quantize images to uint8 if necessary
    image1_8bit = quantize_to_uint8(image1)
    image2_8bit = quantize_to_uint8(image2)

    # Check if the affine transformation is already cached
    if os.path.exists(affine_filename):
        affine_matrix = np.load(affine_filename)
    else:
        # Convert images to grayscale if they are RGB
        if len(image1_8bit.shape) == 3 and image1_8bit.shape[-1] == 3:
            gray1 = cv2.cvtColor(image1_8bit, cv2.COLOR_RGB2GRAY)
        else:
            gray1 = image1_8bit

        if len(image2_8bit.shape) == 3 and image2_8bit.shape[-1] == 3:
            gray2 = cv2.cvtColor(image2_8bit, cv2.COLOR_RGB2GRAY)
        else:
            gray2 = image2_8bit

        # Use ORB to find keypoints and descriptors
        orb = cv2.ORB_create(5000)
        keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
        keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

        # Match descriptors using FLANN-based matcher
        index_params = dict(algorithm=6, table_number=12, key_size=20, multi_probe_level=2)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(descriptors1, descriptors2, k=2)

        # Filter matches using the ratio test
        good_matches = [m for m, n in matches if m.distance < 0.8 * n.distance]

        if len(good_matches) < 3:
            raise ValueError("Not enough good matches to compute affine transformation.")

        # Extract matched points
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Compute the affine transformation
        affine_matrix, _ = cv2.estimateAffinePartial2D(dst_pts, src_pts, method=cv2.RANSAC, ransacReprojThreshold=5.0)

        # Save the affine transformation to disk
        np.save(affine_filename, affine_matrix)

    # Apply the affine transformation
    sz = image1.shape
    aligned_image = cv2.warpAffine(image2, affine_matrix, (sz[1], sz[0]), flags=cv2.INTER_LINEAR)

    return aligned_image

def quantize_to_uint8(image):
    """Convert an image to uint8, scaling values if necessary."""
    if image.dtype != np.uint8:
        if image.max() > 255:
            image = (image / image.max()) * 255
        image = image.astype(np.uint8)
    return image
def create_4_channel_image(green_image, red_image, rededge_image, nir_image):
    """Create a 4-channel image (G, R, RE, NIR)."""
    return np.stack((green_image, red_image, rededge_image, nir_image), axis=-1)
    
def rescale_to_uint16(da):
    da = np.where(np.isnan(da), 65535, da)
    rescaled = (da * 65535).clip(0, 65535).astype(np.uint16)
    
    return rescaled
    
def process_multispec_set(green_image_path, output_directory):
    """Process one set of multispectral images and save as a 4-channel TIFF."""
    base_name = os.path.basename(green_image_path).replace("_G.TIF", "")
    # Load the green band image (reference image)
    green_image, green_metadata = load_image(green_image_path)
    
    bands = {}
    for band in ["R", "RE", "NIR"]:
        band_image_path = green_image_path.replace("_G.TIF", f"_{band}.TIF")
        if os.path.exists(band_image_path):
            bands[band], bands[band+"_meta"] = load_image(band_image_path)
        else:
            print(f"Missing {band} band for {green_image_path}")
            return
        # Construct the output filename MANUALLY CHANGE TO WHATEVER
    sensor = "3M"
    cal = "SUNCAL"
    set_title = os.path.basename(os.path.normpath(output_directory))
    output_filename = f"{sensor}_{cal}_{set_title}_{base_name}.tif"
    output_path = os.path.join(output_directory, output_filename)
    
    # Align each band to the green image using cached homographies
    aligned_red = align_images(green_image, bands["R"], "R", green_image_path, set_title)
    aligned_rededge = align_images(green_image, bands["RE"], "RE", green_image_path, set_title)
    aligned_nir = align_images(green_image, bands["NIR"], "NIR", green_image_path, set_title)

    r_green = calibrate_image(green_image, green_metadata[2]) 

    r_red = calibrate_image(aligned_red, bands["R_meta"][2]) 

    r_rededge = calibrate_image(aligned_rededge, bands["RE_meta"][2]) 

    r_nir = calibrate_image(aligned_nir, bands["NIR_meta"][2]) 

    #rescale to 65535:
    g  = rescale_to_uint16(r_green)
    r = rescale_to_uint16(r_red)
    re = rescale_to_uint16(r_rededge)
    nir = rescale_to_uint16(r_nir)


    # Create a 4-channel image
    image_4ch = create_4_channel_image(g, r, re, nir)


    # Save the 4-channel image
    save_image(output_path, image_4ch,green_metadata)

    rgb_image_path = green_image_path.replace("MS_G.TIF", "D.JPG")
    if os.path.exists(rgb_image_path):
            
        with Image.open(rgb_image_path) as img:
            rgb = np.array(img)
    else:
        print(f"Skipping due to missing jpg path: {rgb_image_path}")
        return
        
    #reszie to fit? (3m is at different res between G and RGB)
    green_shape = green_image.shape
    # Resize RGB to match green image dimensions
    resized_rgb = cv2.resize(rgb, (green_shape[1], green_shape[0]), interpolation=cv2.INTER_AREA)
    aligned_rgb = align_images(green_image, resized_rgb, "RGB", base_name, set_title)
    rgb_filename = f"RGB_{set_title}_{base_name}.tif"
    rgb_path = os.path.join(output_directory, rgb_filename)
    save_image(rgb_path, aligned_rgb,green_metadata)

from concurrent.futures import ThreadPoolExecutor

def process_directory(input_dir, output_directory):
    """Process an entire directory with multiple sets of multispectral images."""
    input_dir = os.path.abspath(input_dir)
    os.makedirs(output_directory, exist_ok=True)

    # Find all green band images (reference images)
    green_images = glob(os.path.join(input_dir, "*_G.TIF"))

    # Process sets of 4 multispectral images in parallel with a progress bar
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        list(tqdm(executor.map(process_multispec_set, green_images, repeat(output_directory)), total=len(green_images)))

    print("All images successfully processed.")



class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})




In [3]:
# Define the number of threads for parallel processing
n_threads = 1
# Define termination criteria for the image registration algorithm; empirically determined:
number_of_iterations = 100
termination_eps = 1e-6

In [5]:
# Create a processed folder if it doesn't exist
processed_dir = "../../data/msrgb_processed/3m_nitrogen/"
os.makedirs(processed_dir, exist_ok=True)
input_folder = "../../data/RGBMS_pretraining/3M_nitrogen"
process_directory(input_folder, processed_dir)

C:\Users\judoj\AppData\Local\Temp\ipykernel_24628\123901474.py:343: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if element:
C:\Users\judoj\AppData\Local\Temp\ipykernel_24628\123901474.py:311: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if element:
 13%|██████████▋                                                                       | 11/84 [00:36<03:56,  3.25s/it]

Skipping due to missing jpg path: C:\Users\judoj\Documents\programming\msdata\data\RGBMS_pretraining\3M_nitrogen\DJI_20230814123356_0011_D.JPG


 27%|██████████████████████▍                                                           | 23/84 [01:18<03:27,  3.40s/it]

Skipping due to missing jpg path: C:\Users\judoj\Documents\programming\msdata\data\RGBMS_pretraining\3M_nitrogen\DJI_20230828111824_0007_D.JPG


 33%|███████████████████████████▎                                                      | 28/84 [01:34<03:06,  3.32s/it]

Skipping due to missing jpg path: C:\Users\judoj\Documents\programming\msdata\data\RGBMS_pretraining\3M_nitrogen\DJI_20230828111842_0012_D.JPG


100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [05:20<00:00,  3.81s/it]

All images successfully processed.


In [4]:
import os
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt

def display_image_bands(folder_path):
    """Load the first 4-channel TIFF image from the folder and display its bands."""
    # Find the first TIFF file in the folder
    tiff_files = [f for f in os.listdir(folder_path) if f.endswith(".tif")]
    if not tiff_files:
        print("No TIFF files found in the folder.")
        return

    first_image_path = os.path.join(folder_path, tiff_files[0])
    
    # Load the image
    image = tiff.imread(first_image_path)
    
    # Validate that it's a 4-channel image
    if image.shape[-1] != 4:
        print("The image does not have 4 channels.")
        return
    
    # Extract the bands
    green_band = image[:, :, 0]
    red_band = image[:, :, 1]
    rededge_band = image[:, :, 2]
    nir_band = image[:, :, 3]
    
    # Create a false-color image (NIR, Red, Green)
    false_color_image = np.dstack((nir_band, red_band, green_band))
    false_color_image = false_color_image / 65535

    # Create a 10x zoom of the center crop
    height, width, _ = false_color_image.shape
    crop_size = (height // 20, width // 20)  # Crop size for a 10x zoom
    center_y, center_x = height // 2, width // 2

    # Get the cropped region
    cropped_image = false_color_image[
        center_y - crop_size[0] // 2 : center_y + crop_size[0] // 2,
        center_x - crop_size[1] // 2 : center_x + crop_size[1] // 2
    ]

    # Resize the cropped image to match the original dimensions
    zoomed_image = cv2.resize(cropped_image, (width, height), interpolation=cv2.INTER_LINEAR)

    # Plot the images
    fig, ax = plt.subplots(2, 3, figsize=(18, 12))

    # False color image
    ax[0, 0].imshow(false_color_image)
    ax[0, 0].set_title("False Color (NIR, Red, Green)")
    ax[0, 0].axis("off")

    # Individual bands
    ax[0, 1].imshow(green_band, cmap="gray")
    ax[0, 1].set_title("Green Band")
    ax[0, 1].axis("off")

    ax[0, 2].imshow(red_band, cmap="gray")
    ax[0, 2].set_title("Red Band")
    ax[0, 2].axis("off")

    ax[1, 0].imshow(rededge_band, cmap="gray")
    ax[1, 0].set_title("RedEdge Band")
    ax[1, 0].axis("off")

    ax[1, 1].imshow(nir_band, cmap="gray")
    ax[1, 1].set_title("NIR Band")
    ax[1, 1].axis("off")

    # Zoomed-in section
    ax[1, 2].imshow(zoomed_image)
    ax[1, 2].set_title("10x Zoom (Center Crop)")
    ax[1, 2].axis("off")

    plt.tight_layout()
    plt.show()


In [5]:

display_image_bands("../data/processed/3m_nitrogen")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/processed/3m_nitrogen'